In [2]:
import pandas as pd
from smartsheet_dataframe import get_as_df
import os

In [3]:
# Storing Smartsheet information in variables
shitz_token = "wMvGniITjDFd7ClZPE44wtJGvZkM7Hg7mP7if" #Bily's personal token to access G-A smartsheet data. Anyone can obtain their own token
fhaid = 4523924143794052 # The ID number that references the FHA sheet
mssid = 3005704744265604 # The ID number that references the MSS sheet

# Creating the FHA and MSS dataframes
fha_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=fhaid)

mss_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=mssid)

bjt_df = mss_df.loc[mss_df['UniqueID'].str.contains("BJT")].copy()

In [7]:
client_df = bjt_df[['UniqueID',
                    'Tract',
                    'CMP',
                    'Stand',
                    'Acres',
                    'Age',
                    'County',
                    'State',
                    'Stand Type',
                    'Origin',
                    'Scheduled',
                    'PreBA',
                    'PreTPA',
                    'PreGTA',
                    'PlnBA',
                    'PlnTPA',
                    'PlnGTA',
                    'Client GT',
                    'Future GT',
                    'Reporting Period']].copy()

client_df['Reporting Period'] = client_df['Reporting Period'].fillna(0)
client_df.loc[client_df['Reporting Period'] == "", 'Reporting Period'] = 0
#s = client_df.loc[pd.to_numeric(client_df['Reporting Period'], errors='coerce').isnull() & client_df['Reporting Period'].str.contains("RP"), 'Reporting Period'].apply(lambda x: x.strip()[-1])
#client_df['trial'] = client_df.index.map(s)
#client_df.loc[pd.to_numeric(client_df['Reporting Period'], errors='coerce').isnull() & client_df['Reporting Period'].str.contains("RP"), 'Reporting Period'] = client_df.loc[pd.to_numeric(client_df['Reporting Period'], errors='coerce').isnull() & client_df['Reporting Period'].str.contains("RP"), 'trial']
client_df['Reporting Period'] = pd.to_numeric(client_df['Reporting Period'], downcast='integer')
client_df['Age'] = pd.to_numeric(client_df['Age'], errors='coerce', downcast='integer').fillna(0).astype(int)
client_df.loc[pd.to_numeric(client_df['Future GT'], errors='coerce').isnull(), 'Future GT'] = 0
client_df['Tract'] = client_df['Tract'].astype(str)
client_df['Stand'] = client_df['Stand'].astype(str)
client_df['Tract'] = client_df['Tract'].apply(lambda x: x.split('.')[0])
client_df['Client GT'] = pd.to_numeric(client_df['Client GT'], errors="coerce")
client_df['Client GT'] = client_df['Client GT'].fillna(0)

cols = ['PreBA', 'PreTPA', 'PreGTA', 'PlnBA', 'PlnTPA', 'PlnGTA']

for x in cols:
    client_df[x] = pd.to_numeric(client_df[x], errors='coerce', downcast='integer').fillna(-99).astype(int)

#client_df.drop(columns=['trial'],inplace=True)


In [9]:
species = ["Slash Pine", "Loblolly Pine", 'Mixed Natural Pine', 'Hardwood Mix', 'Pine Straw', "Other"]
species_dict = {'Species': species}
species_df = pd.DataFrame(species_dict)
species_df['SpeciesUID'] = species_df.index

def species_map(x):
    x = x.lower()
    if "slash" in x:
        return 0
    elif "loblol" in x:
        return 1
    elif "mixed natural" in x:
        return 2
    elif "h" in x:
        return 3
    elif "straw" in x:
        return 4
    elif "pine" in x:
        return 1
    else:
        return 5
    
client_df['Stand Type'] = client_df['Stand Type'].apply(lambda x: species_map(x))


origins = ['Planted', 'Natural', 'Natural Mix', 'Other']
origin_dict = {'Origin': origins}
origin_df = pd.DataFrame(origin_dict)
origin_df['OriginUID'] = origin_df.index

def origin_map(x):
    x = x.lower()
    if "plant" in x:
        return 0
    elif "natural mix" in x:
        return 2
    elif "nat" in x:
        return 1
    else:
        return 3
client_df['Origin'] = client_df['Origin'].apply(lambda x: origin_map(x))


In [10]:
ga_intake_df = bjt_df[['IntakeID',
                       'FolderID',
                       'UniqueID',
                       'GA_ACRES',
                       'GA GT', # this is going to be GA total isn't it? Or does it stay as GA estimate of current GT (total minus what client says is future)
                       'Total GT', # Does this need included? At the moment we do funny stuff with total gt and subtracting future gt to get GA current rp (above)
                       'Notes'
                       ]].copy()

# Do I want to include a Ticketid column here or use a crossreference table?
# Should reporting period be in here?

In [11]:
status_df = bjt_df[['UniqueID',
                    'Guidance',
                    'Expiration',
                    'GuidanceProvider',
                    'Status',
                    'Completed Date'
                    ]].copy()

In [12]:
def intake_breaker(lst):
    intake_lst = []
    activity_lst = []
    for ele in lst:
        intake_lst.append(ele[:9])
        activity_lst.append(ele[9:11])
        intake_st = set(intake_lst)
        activity_st = set(activity_lst)
        
    return intake_lst, activity_lst, intake_st, activity_st


def ticketid(lst):
    """This function creates a ticket id based off a list of unique ids

    Args:
        lst (list): a list of unique ids that have the same Tract_Cmp_Stand from MSS
    """
    if len(lst) == 0:
        return "ERROR, Where is your uniqueid?"
    elif len(lst) < 2:
        return lst[0]
    elif len(lst) > 1:
        # Check to see if there are multiple intakes
        temp_lst, activity_lst, temp_st, activity_st = intake_breaker(lst)
        temp_lst.sort()
        activity_lst.sort()
        if len(temp_st) > 1:
            return f"{temp_lst[0]}XXXX"
        # If there are not multiple intakes. Check to see if there are multiple activities in the lst (Intake portion should be same for all)
        elif len(activity_st) > 1:
           return f"{temp_lst[0]}XXXX"
       # If there is only a single activity type
        elif len(activity_st) == 1:
            return f"{temp_lst[0]}{activity_lst[0]}XX"
        else:
            return "Failed within len(lst) > 1"
        
    else:
        return "Failed at length of lst" 

In [19]:
bjt_inprog = bjt_df[bjt_df['Status'].isin(['OPEN', 'IN PROCESS', 'POST', 'PENDING'])].copy()
bjt_inprog['Tract'] = bjt_inprog['Tract'].astype(str).apply(lambda x: x.split('.')[0])
bjt_inprog['CMP'] = bjt_inprog['CMP'].astype(str).apply(lambda x: x.split('.')[0])
bjt_inprog['Stand'] = bjt_inprog['Stand'].astype(str).apply(lambda x: x.split('.')[0])
bjt_inprog['TCS'] = bjt_inprog['Tract'] + "_" + bjt_inprog['CMP'] + "_" + bjt_inprog['Stand']

# After talking with Rob, we need to make sure that we separate out the GA and the ARB Guidance
grouped_activities = bjt_inprog.groupby(['TCS', 'GuidanceProvider'])['UniqueID'].apply(list).reset_index()

trial_df = grouped_activities.copy()
trial_df['BBID'] = trial_df['UniqueID'].apply(lambda x: ticketid(x))

prev_combined = bjt_inprog.groupby('TicketID')['TCS'].apply(set).reset_index()

tcs = []
for val in prev_combined['TCS'].loc[prev_combined['TCS'].str.len()> 1].values:
    for item in val:
        tcs.append(item)
        
ticket_id_dict = {}
for i, row in prev_combined.loc[prev_combined['TCS'].str.len()> 1].iterrows():
    # print(row['TicketID']) # ticket id
    value_list = []
    for value in row['TCS']:
        value_list.append(value)
    ticket_id_dict[row['TicketID']] = value_list
    
if len(ticket_id_dict) > 0:    
    for k, v in ticket_id_dict.items():
        trial_df.loc[trial_df['TCS'].isin(v), 'otherid'] = k
else:
    trial_df['otherid'] = "nan"

def final_id(row):
    if row['otherid'] != "nan":
        return row['otherid']
    else:
        return row['BBID']
    
trial_df['finalid'] = trial_df[['BBID', 'otherid']].apply(final_id, axis=1)

def break_list(x):
    trial_list = []
    for item in x:
        for iter in item:
            trial_list.append(iter)
            
    return trial_list

new_ticket_df = trial_df.groupby('finalid').agg({'TCS': lambda x: list(x),
                                 'UniqueID': lambda x: break_list(x)}).reset_index()

client_for_ticket = client_df.copy()
for id in new_ticket_df['finalid'].unique().tolist():
    client_for_ticket.loc[client_for_ticket['UniqueID'].isin(new_ticket_df.loc[new_ticket_df['finalid'] == id, 'UniqueID'].values[0]), 'TicketID'] = id

new_tcs_intake = client_for_ticket.groupby(['TicketID',
                                            'UniqueID'])['Client GT'].sum().reset_index()

old_ticket_ids = dict(zip(bjt_df['UniqueID'], bjt_df['TicketID']))
new_tcs_intake['old_ticketids'] = new_tcs_intake['UniqueID'].map(old_ticket_ids)
new_tcs_intake.loc[~new_tcs_intake['TicketID'].str.contains("BJT"), 'TicketID'] = new_tcs_intake['old_ticketids']


ticketid_table = new_tcs_intake[['TicketID', 'UniqueID']]

In [23]:
mill_loc = r"P:\Mill Tickets"
file_pattern = "BJT_RP*_Compiled.xlsm"
big_df = pd.DataFrame()

mill_name = []
for root, dirs, files in os.walk(mill_loc):
    for file in files:
        if "BJT_RP" in file and file.endswith("Compiled.xlsm"):
            if "$" in file:
                pass
            else:
                print(root)
                print(file)
                df = pd.read_excel(os.path.join(root, file), sheet_name='RP_Project')
                big_df = pd.concat([big_df, df])
                

# big_df.loc[big_df['Tons (U.S)'].isnull(), 'Tons (U.S)'] = big_df.loc[big_df['Tons (U.S)'].isnull(), 'TonsUS']
# big_df['Cull Tons'].fillna(big_df['CullTons'], inplace=True)
# big_df['Accepted Tons'].fillna(big_df['AcceptedTons'], inplace=True)
# big_df['Reporting Period'].fillna(big_df['ReportingPeriod'],inplace=True)

def rep_fix(x):
    if "RP" in str(x):
        return x[-1]
    else:
        return x
    
big_df['Reporting Period'] = big_df['Reporting Period'].apply(lambda x: rep_fix(x))
big_df['Reporting Period'] = pd.to_numeric(big_df['Reporting Period'])

big_df['Date'] = pd.to_datetime(big_df['Date'])

mill_df = big_df[['Date',
                  'Ticket',
                  'Mill',
                  'Product',
                  'Tons (U.S)',
                  'Cull Tons',
                  'Accepted Tons',
                  'Source',
                  'Notes',
                  'Reporting Period',
                  'TicketID',
                  'Week']].copy()

mill_df = mill_df.drop_duplicates()

P:\Mill Tickets\ARCHIVE\BJT_RP5
BJT_RP5_Compiled.xlsm
P:\Mill Tickets\ARCHIVE\BJT_RP6
BJT_RP6_Compiled.xlsm
P:\Mill Tickets\ARCHIVE\BJT_RP7
BJT_RP7_Compiled.xlsm


C:\Users\bbrown\AppData\Local\Temp\ipykernel_22752\2166520941.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  big_df = pd.concat([big_df, df])


P:\Mill Tickets\BJT_RP8
BJT_RP8_Compiled.xlsm


There was no harvesting in the previous reporting period

In [30]:
# Getting harvest ids that were in RP 8 and are still "Active" (Potentially need updating due to harvest)
harvest_activities = bjt_df[(bjt_df['Reporting Period'] == 7) & (bjt_df['Status'].isin(['IN PROCESS', "POST"]))]['UniqueID'].unique().tolist()
harvest_old_ticketid = bjt_df[(bjt_df['Reporting Period'] == 7) & (bjt_df['Status'].isin(['IN PROCESS', "POST"]))]['TicketID'].unique().tolist()

In [31]:
# Let's get the new TIDs based on these harvest activities
new_ticketIDs = ticketid_table[ticketid_table['UniqueID'].isin(harvest_activities)]['TicketID'].unique().tolist()


In [32]:
new_ticketIDs

['BJT230510TT01']

This should be easier to get the expected for the current activities - since there is no accounting that needs to be done.

In [36]:
bjt_inprocess = bjt_df[bjt_df['Status'].isin(['IN PROCESS', 'OPEN', 'POST', 'PENDING'])].copy()
bjt_inprocess = bjt_inprocess[['UniqueID', 'TicketID', 'Reporting Period', 'Current RP Expected GT', 'Future GT', 'Total GT']]
bjt_inprocess

,UniqueID,TicketID,Reporting Period,Current RP Expected GT,Future GT,Total GT
18,BJT241022CC02,BJT241022CCXX,8.0,1945.94,0.0,1945.94
19,BJT241022CC01,BJT241022CCXX,8.0,2554.68,0.0,2554.68
60,BJT240814TH01,BJT240814TH01,8.0,1672.65,0.0,1672.65
694,BJT230727CC01,BJT230727CC01,7.0,3195.69,0.0,3195.69
695,BJT230727CC05,BJT230727CC05,7.0,3061.29,0.0,3061.29
696,BJT230727CC04,BJT230727CC04,7.0,350.33,0.0,350.33
697,BJT230727CC03,BJT230727CC03,7.0,1041.7,0.0,1041.7
698,BJT230727CC02,BJT230727CC02,7.0,169.41,0.0,169.41
699,BJT230510TT01,BJT230510TT01,7.0,8191.0,0.0,8191.47


In [37]:
bjt_7_ageup = bjt_inprocess[bjt_inprocess['Reporting Period'] == 7.0].copy()
bjt_7_ageup['Reporting Period'] = 8
bjt_inprocess = pd.concat([bjt_inprocess, bjt_7_ageup])
bjt_inprocess

,UniqueID,TicketID,Reporting Period,Current RP Expected GT,Future GT,Total GT
18,BJT241022CC02,BJT241022CCXX,8.0,1945.94,0.0,1945.94
19,BJT241022CC01,BJT241022CCXX,8.0,2554.68,0.0,2554.68
60,BJT240814TH01,BJT240814TH01,8.0,1672.65,0.0,1672.65
694,BJT230727CC01,BJT230727CC01,7.0,3195.69,0.0,3195.69
695,BJT230727CC05,BJT230727CC05,7.0,3061.29,0.0,3061.29
696,BJT230727CC04,BJT230727CC04,7.0,350.33,0.0,350.33
697,BJT230727CC03,BJT230727CC03,7.0,1041.7,0.0,1041.7
698,BJT230727CC02,BJT230727CC02,7.0,169.41,0.0,169.41
699,BJT230510TT01,BJT230510TT01,7.0,8191.0,0.0,8191.47
694,BJT230727CC01,BJT230727CC01,8,3195.69,0.0,3195.69


In [ ]:
bjt_inprocess = pd.merge(bjt_inprocess, new_ticket_df, left_on='TicketID', right_on='finalid', how='left')


In [41]:
bjt_inprocess

,UniqueID_x,TicketID,Reporting Period,Current RP Expected GT,Future GT,Total GT,finalid,TCS,UniqueID_y
0,BJT241022CC02,BJT241022CCXX,8.0,1945.94,0.0,1945.94,BJT241022CCXX,[BJT 39_NP_1],"[BJT241022CC02, BJT241022CC01]"
1,BJT241022CC01,BJT241022CCXX,8.0,2554.68,0.0,2554.68,BJT241022CCXX,[BJT 39_NP_1],"[BJT241022CC02, BJT241022CC01]"
2,BJT240814TH01,BJT240814TH01,8.0,1672.65,0.0,1672.65,BJT240814TH01,[BJT_122_17],[BJT240814TH01]
3,BJT230727CC01,BJT230727CC01,7.0,3195.69,0.0,3195.69,BJT230727CC01,[BJT_122_22],[BJT230727CC01]
4,BJT230727CC05,BJT230727CC05,7.0,3061.29,0.0,3061.29,BJT230727CC05,[BJT_122_13],[BJT230727CC05]
5,BJT230727CC04,BJT230727CC04,7.0,350.33,0.0,350.33,BJT230727CC04,[BJT_122_23],[BJT230727CC04]
6,BJT230727CC03,BJT230727CC03,7.0,1041.7,0.0,1041.7,BJT230727CC03,[BJT_122_19],[BJT230727CC03]
7,BJT230727CC02,BJT230727CC02,7.0,169.41,0.0,169.41,BJT230727CC02,[BJT_122_16],[BJT230727CC02]
8,BJT230510TT01,BJT230510TT01,7.0,8191.0,0.0,8191.47,BJT230510TT01,[39_NP_3],[BJT230510TT01]
9,BJT230727CC01,BJT230727CC01,8,3195.69,0.0,3195.69,BJT230727CC01,[BJT_122_22],[BJT230727CC01]


In [47]:
bjt_inprocess[bjt_inprocess['Reporting Period'] > 7].groupby(['TicketID']).agg({'TCS': "first",
                                                                                "Total GT": "sum"}).reset_index()

,TicketID,TCS,Total GT
0,BJT230510TT01,[39_NP_3],8191.47
1,BJT230727CC01,[BJT_122_22],3195.69
2,BJT230727CC02,[BJT_122_16],169.41
3,BJT230727CC03,[BJT_122_19],1041.7
4,BJT230727CC04,[BJT_122_23],350.33
5,BJT230727CC05,[BJT_122_13],3061.29
6,BJT240814TH01,[BJT_122_17],1672.65
7,BJT241022CCXX,[BJT 39_NP_1],4500.62


In [52]:
bjt_inprocess['tcsclean?'] = bjt_inprocess['TCS'].apply(lambda x: x[0])

In [55]:
bjt_client = bjt_inprocess[bjt_inprocess['Reporting Period'] > 7].groupby(['TicketID']).agg({'tcsclean?': "first",
                                                                                            "Total GT": "sum"}).reset_index()
bjt_client.to_excel('bjt_client_intake.xlsx')

In [43]:
bjt_inprocess.groupby(['TicketID', 'Reporting Period'])['Total GT'].sum()

TicketID       Reporting Period
BJT230510TT01  7.0                 8191.47
               8.0                 8191.47
BJT230727CC01  7.0                 3195.69
               8.0                 3195.69
BJT230727CC02  7.0                  169.41
               8.0                  169.41
BJT230727CC03  7.0                  1041.7
               8.0                  1041.7
BJT230727CC04  7.0                  350.33
               8.0                  350.33
BJT230727CC05  7.0                 3061.29
               8.0                 3061.29
BJT240814TH01  8.0                 1672.65
BJT241022CCXX  8.0                 4500.62
Name: Total GT, dtype: object